In [207]:
from aipython.stripsProblem import STRIPS_domain, Planning_problem
from aipython.stripsForwardPlanner import Forward_STRIPS
from aipython.searchMPP import SearcherMPP
from lab02.aipython.stripsForwardPlanner import State


In [208]:
from lab02.aipython.stripsProblem import Strips

class Strips_starcraft(object):
    def __init__(self, name, preconds, add_effects, del_effects=None, cost=1):
        """
        Defines the STRIPS representation for an action:
        - name is the name of the action
        - preconds, the preconditions, is a dictionary {feature: value} that must hold for the action to be carried out
        - add_effects is a dictionary {feature: value} of the positive effects of the action
        - del_effects is an optional dictionary {feature: value} of the negative effects of the action
        - cost is the cost of the action
        """
        self.name = name
        self.preconds = preconds
        self.add_effects = add_effects
        self.del_effects = del_effects if del_effects is not None else {}
        self.cost = cost

    def __repr__(self):
        return self.name

# Define the STRIPS domain
delivery_domain = STRIPS_domain(
    {
        'RLoc': {'cs', 'off', 'lab', 'mr'},  # Possible robot locations
        'RHC': {False, True},  # Robot has cargo?
        'SWC': {False, True},  # Warehouse has cargo?
        'MW': {False, True},   # Robot has maintenance need?
        'RHM': {False, True}   # Robot has maintenance mode?
    },
    {  # Set of STRIPS actions
        Strips('mc_cs', {'RLoc': 'cs'}, {'RLoc': 'off'}),
        Strips('mc_off', {'RLoc': 'off'}, {'RLoc': 'lab'}),
        Strips('mc_lab', {'RLoc': 'lab'}, {'RLoc': 'mr'}),
        Strips('mc_mr', {'RLoc': 'mr'}, {'RLoc': 'cs'}),
        Strips('mcc_cs', {'RLoc': 'cs'}, {'RLoc': 'mr'}),
        Strips('mcc_off', {'RLoc': 'off'}, {'RLoc': 'cs'}),
        Strips('mcc_lab', {'RLoc': 'lab'}, {'RLoc': 'off'}),
        Strips('mcc_mr', {'RLoc': 'mr'}, {'RLoc': 'lab'}),
        Strips('puc', {'RLoc': 'cs', 'RHC': False}, {'RHC': True}),
        Strips('dc', {'RLoc': 'off', 'RHC': True}, {'RHC': False, 'SWC': False}),
        Strips('pum', {'RLoc': 'mr', 'MW': True}, {'RHM': True, 'MW': False}),
        Strips('dm', {'RLoc': 'off', 'RHM': True}, {'RHM': False})
    }
)


def heuristic(state, goal):
    """
    Heuristic function to estimate the cost to reach the goal state from the given state.
    """
    # Calculate the number of unsatisfied goals
    unsatisfied_goals = 0
    for key, value in goal.items():
        if state[key] != value:
            unsatisfied_goals += 1
    return unsatisfied_goals

def format_path(path):
    """
    Format the path for readability.
    """
    formatted_steps = []
    for i, step in enumerate(path.steps):
        state = step.state
        action = step.action
        if i == 0:
            formatted_steps.append(f"Initial State: {state}")
        else:
            formatted_steps.append(f"Step {i}: Perform action '{action}'")
            formatted_steps.append(f"Resulting State: {state}")
    return "\n".join(formatted_steps)


## Problem 1

In [209]:
# Define the initial state
initial_state = {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}

# Define the goal state
goal_state = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj = State(initial_state)
goal_state_obj = State(goal_state)

# Calculate the heuristic value using the heuristic function and initial state
# heuristic_value = heuristic(initial_state_obj, goal_state_obj)

# Define the Planning problem
problem_1 = Planning_problem(delivery_domain, initial_state, goal_state)

# Instantiate the Forward_STRIPS class with the planning problem and heuristic function
forward_strips = Forward_STRIPS(problem_1)


# Use A* search algorithm to find a solution
solution = SearcherMPP(forward_strips).search()

# Print the solution
print(solution)

Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --puc--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --dc--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 7)
 27 paths have been expanded and 16 paths remain in the frontier
{'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SW

## Problem 2

In [210]:
initial_state_2 = { 'RLoc': 'lab', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False }

goal_state_2 = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj_2 = State(initial_state)
goal_state_obj_2 = State(goal_state)

# Define the Planning problem
problem_2 = Planning_problem(delivery_domain, initial_state_2, goal_state_2)

# Solve the problem using the forward planner
forward_strips_2 = Forward_STRIPS(problem_2, heur=heuristic)
solution_2 = SearcherMPP(forward_strips_2).search()

    

Solution: {'RLoc': 'lab', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': False} (cost: 5)
 12 paths have been expanded and 10 paths remain in the frontier


## Problem 3

In [211]:
# Define the initial state and goal state
initial_state_3 = { 'RLoc': 'lab', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True }


goal_state = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj_3 = State(initial_state)
goal_state_obj_3 = State(goal_state)

# Define the Planning problem
problem_3 = Planning_problem(delivery_domain, initial_state_3, goal_state)

# Solve the problem
forward_strips_3 = Forward_STRIPS(problem_3)
solution_3 = SearcherMPP(forward_strips_3).search()


Solution: {'RLoc': 'lab', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True}
   --mc_lab--> {'RLoc': 'mr', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True}
   --pum--> {'RLoc': 'mr', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --dc--> {'RLoc': 'off', 'RHC': False, 'SWC': False, 'MW': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'RHC': False, 'SWC': False, 'MW': False, 'RHM': False} (cost: 6)
 28 paths have been expanded and 19 paths remain in the frontier


# Harder problem

In [217]:
def gen_starcraft_move(from_loc, to_loc, unit):
    return 'move-' + from_loc + '-' + to_loc + '-' + unit


def gen_starcraft_collect_minerals(area, unit):
    return 'collect-minerals-' + area + '-' + unit


def gen_starcraft_build(unit, area, building):
    return 'build-' + unit + '-' + area + '-' + building


def gen_starcraft_train_marine(builder, building, area):
    return 'train-marine-' + builder + '-' + building + '-' + area

def at(x, y):
    return '' + x + '-at-' + y

def min_depleted(x):
    return '' + x + '-empty'

def has_minerals(x):
    return '' + x + '-has-minerals'

def building(x):
    return 'building-at-' + x


def build_starcraft_domain(builder=None, building=None, area=None):
    if area is None:
        area = {'sectorA', 'sectorB', 'mineralFieldA', 'mineralFieldB', 'mineralFieldC'}
    if building is None:
        building = {'barracks', 'factory', 'starport', 'depot', 'empty'}
    if builder is None:
        builder = {'scv', 'marine', 'tank', 'battlecruiser'}
        
    # MOVE
    stmap = {
        Strips(
            gen_starcraft_move(x, y, z),
            {at(z, x): True, at(z, y): False},
            {at(z, x): False, at(z, y): True}
               )
        for x in area 
        for y in area 
        for z in builder
        if x != y
        if z == 'scv'
    }
    # COLLECT MINERALS
    stmap.update({
        Strips(
            gen_starcraft_collect_minerals(x, z),
            {at(z, x): True, min_depleted(x): False},
            {min_depleted(x): True, has_minerals(z): True}
        )
        for x in area for z in builder
        if z == 'scv'
    })
    # BUILD
    stmap.update({
        Strips(
            gen_starcraft_build(x, y, z),
            {at(x, y): True, min_depleted(y): True, has_minerals(x): True, building(y): 'empty'},
            {building(y): 'depot', has_minerals(x): False}
        )
        for x in builder 
        for y in area
        for z in building
        if z != 'empty'
        if x == 'scv'
    })
    # TRAIN MARINE
    stmap.update({
        Strips(
            gen_starcraft_train_marine('scv', 'barracks', z),
            {building(z): 'barracks', has_minerals('scv'): True},
            {has_minerals('scv'): False, at('marine', z): True}
        )
        for z in area
    })
    # TRAIN TANK
    stmap.update({
        Strips(
            gen_starcraft_train_marine('scv', 'factory', z),
            {building(z): 'factory', has_minerals('scv'): True},
            {has_minerals('scv'): False, at('tank', z): True}
        )
        for z in area
    })
    # TRAIN BATTLECRUISER 
    stmap.update({
        Strips(
            gen_starcraft_train_marine('scv', 'starport', z),
            {building(z): 'starport', has_minerals('scv'): True},
            {has_minerals('scv'): False, at('battlecruiser', z): True}
        )
        for z in area
    })
    
    feature_domain_dict = {
        
    
    fea
    
    return STRIPS_domain(feature_domain_dict, stmap)


starcraft_domain = build_starcraft_domain()


TypeError: 'set' object is not callable

In [216]:
# Starcraft Problem 1
# Train marine

# Assuming State class is defined elsewhere and is capable of handling the initial and goal states as defined.
initial_state_marine = {
    occupy_area('sectorA', 'scv'),
    clear_area('sectorB'),
    clear_area('mineralFieldA'),
    clear_area('mineralFieldB'),
    
}

# Goal State
goal_state_marine = {
    gen_at_area('sectorA', 'marine')
}

# Assuming the Planning_problem class correctly initializes a problem given a domain, initial state, and goal state
problem_4 = Planning_problem(starcraft_domain, initial_state_marine, goal_state_marine)

# Assuming Forward_STRIPS and SearcherMPP are part of your planning framework
# Forward_STRIPS should prepare the problem for forward search
forward_strips_4 = Forward_STRIPS(problem_4)

# SearcherMPP is assumed to be a multi-path planner that searches for solutions
solution_4 = SearcherMPP(forward_strips_4).search()

# Make sure that you have the implementations for State, Forward_STRIPS, and SearcherMPP classes or functions
# This code assumes those implementations are part of your environment or external libraries being used.

AttributeError: 'set' object has no attribute 'items'

In [214]:
# Starcraft Problem 2
# build tank

initial_state_tank = {
    'Builder': frozenset({'scv'}),
    'Building': frozenset(),
    'Area': frozenset({'sectorA', 'sectorB', 'mineralFieldA', 'mineralFieldB', 'mineralFieldC'}),
    'Predicate': frozenset([
        ('scv', 'scv'),
        ('location', 'sectorA'),
        ('location', 'sectorB'),
        ('location', 'mineralFieldA'),
        ('location', 'mineralFieldB'),
        ('location', 'mineralFieldC'),
        ('minerals', 'mineralFieldA'),
        ('minerals', 'mineralFieldB'),
        ('minerals', 'mineralFieldC'),
        ('at', ('scv', 'sectorA'))
    ])
}

goal_state_tank = {
    'Predicate': frozenset([('tank', 'sectorA')])
}

initial_state_5 = State(initial_state_tank)
goal_state_5 = State(goal_state_tank)

problem_5 = Planning_problem(starcraft_domain, initial_state_tank, goal_state_tank)

forward_strips_5 = Forward_STRIPS(problem_5)

solution_5 = SearcherMPP(forward_strips_5).search()


KeyError: 'at-barracks-mineralFieldA'

In [215]:
# Starcraft Problem 3
# build battlecruiser

initial_state_battlecruiser = {
    'Builder': frozenset({'scv'}),
    'Building': frozenset(),
    'Area': frozenset({'sectorA', 'sectorB', 'mineralFieldA', 'mineralFieldB', 'mineralFieldC'}),
    'Predicate': frozenset([
        ('scv', 'scv'),
        ('location', 'sectorA'),
        ('location', 'sectorB'),
        ('location', 'mineralFieldA'),
        ('location', 'mineralFieldB'),
        ('location', 'mineralFieldC'),
        ('minerals', 'mineralFieldA'),
        ('minerals', 'mineralFieldB'),
        ('minerals', 'mineralFieldC'),
        ('at', ('scv', 'sectorA'))
    ])
}

goal_state_battlecruiser = {
    'Predicate': frozenset([('battlecruiser', 'sectorA')])
}

initial_state_6 = State(initial_state_battlecruiser)
goal_state_6 = State(goal_state_battlecruiser)

problem_6 = Planning_problem(starcraft_domain, initial_state_battlecruiser, goal_state_battlecruiser)

forward_strips_6 = Forward_STRIPS(problem_6)

solution_6 = SearcherMPP(forward_strips_6).search()

KeyError: 'at-barracks-mineralFieldA'